### Import libraries

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet, Title

output_notebook()

Loading BokehJS ...

### Our goal

<img src="../../images/figure_2.png" width="50%" height="50%">

#### Colors 

In [2]:
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2, BLUE3, BLUE4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
RED1, RED2 = '#C3514E', '#E6BAB7'
GREEN1, GREEN2 = '#0C8040', '#9ABB59'
ORANGE1 = '#F79747'

#### Font 

In [3]:
FONT = 'Arial'

#### Preprocessing 

In [4]:
dataset = pd.read_csv('../../data/data_ticket_volumen_over_time.csv', index_col='State')
dataset.shape

(2, 12)

In [5]:
dataset.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
State,,,,,,,,,,,,
Received,160,184,241,149,180,161,132,202,160,139,149,177
Processed,160,184,237,148,181,150,123,156,126,104,124,140


In [6]:
dataset = dataset.rename_axis(None).T.reset_index().rename(columns={'index': 'Month'})

In [7]:
dataset.head()

,Month,Received,Processed
0,Jan,160,160
1,Feb,184,184
2,Mar,241,237
3,Apr,149,148
4,May,180,181


In [8]:
dataset.head(1)

,Month,Received,Processed
0,Jan,160,160


In [9]:
data = []

new_row = {'Month':'    ', 'Received':np.nan, 'Processed':np.nan}
data.insert(0, new_row)

data = pd.DataFrame(data)[dataset.columns]
dataset = pd.concat([pd.DataFrame(data), dataset], ignore_index=True, axis=0)

In [10]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
# This code adds extra space on the right
new_row = {'Month':''}
dataset = dataset.append(new_row, ignore_index=True)

new_row = {'Month':' '}
dataset = dataset.append(new_row, ignore_index=True)

new_row = {'Month':'  '}
dataset = dataset.append(new_row, ignore_index=True)

#### Plot 

In [29]:
# Set the source of the plot
source = ColumnDataSource(dataset)
cut_source = ColumnDataSource(dataset[8:]) # Add labels from Agust fordward


# Create the figure
p = figure(x_range=list(dataset['Month']), 
           y_range=(-30, 310), 
           plot_height=600, 
           plot_width=830, 
           title='Please approve the hire of 2 FTEs',
           toolbar_location='above')


# Add lines and circles to the figure
p.line(x='Month', 
       y='Received', 
       line_color=GRAY6,
       line_width=5,
       source=source)

p.circle(x='Month', 
         y='Received', 
         color=GRAY6, 
         size=12, 
         source=cut_source)

p.line(x='Month', 
       y='Processed', 
       line_color=BLUE1, 
       line_width=5,
       source=source)

p.circle(x='Month', 
         y='Processed', 
         color=BLUE1, 
         size=12, 
         source=cut_source)


# Add legends on the lines
p.add_layout(LabelSet(x='Month', 
                      y='Processed',                      
                      y_offset=-30,           
                      text=str('Processed'), 
                      text_align='center',                      
                      text_baseline='bottom', 
                      text_color=BLUE1, 
                      text_font=FONT,
                      text_font_size='13pt',
                      source=cut_source))

p.add_layout(LabelSet(x='Month', 
                      y='Received', 
                      y_offset=15, 
                      text=str('Received'), 
                      text_align='center',                      
                      text_baseline='bottom', 
                      text_color=GRAY6,
                      text_font=FONT,
                      text_font_size='13pt',
                      source=cut_source))


# Add subtitles
p.add_layout(Title(text=" "), "above") # add extra space between title and subtitle

p.add_layout(Title(text="Ticket volumen over time", 
                   align="left", 
                   offset=0,                                   
                   text_color=GRAY4,
                   text_font=FONT,   
                   text_font_size='17pt',
                   text_font_style='normal'), 
             "above")

p.add_layout(Title(text="to backfill those who quit in the past year", 
                   align="left", 
                   offset=0,                    
                   text_color=GRAY7,
                   text_font=FONT,
                   text_font_size='14pt',
                   text_font_style='normal'), 
             "above")


# Add text annotations
# This is a weakness of bokeh, you cannot format a single word within a text. We have to do it separately.

# Add legends annotations
p.text(x=13, 
       y=125, 
       text=['Processed'],       
       text_color=BLUE1, 
       text_font=FONT,
       text_font_size='17pt', 
       text_font_style='bold')

p.text(x=13, 
       y=175, 
       text=['Received'],       
       text_color=GRAY6, 
       text_font=FONT,
       text_font_size='17pt', 
       text_font_style='bold')

# Text annotation
p.text(x=4.6, 
       y=283, 
       text=['2 employees quit in May.'],
       text_color=GRAY5,
       text_font=FONT,
       text_font_size='12.5pt', 
       text_font_style='bold')

p.text(x=4.6, 
       y=237, 
       text=['\t' * 42 + ' We nearly kept up with incoming \nvolume in the following two months but fell behind with the \nincrease in Aug and have\'t been able to catch up since.'],
       text_color=GRAY5,
       text_font=FONT,
       text_font_size='12.5pt')

p.segment(x0=4.5, 
          y0=0, 
          x1=4.5,
          y1=240, 
          color=GRAY4, 
          line_width=1)


# Add top legend
p.text(x=0.5, 
       y=197,
       x_offset=0,
       y_offset=0,
       angle=90,
       angle_units='deg',
       text=['Number of tickets'],
       text_font=FONT,
       text_font_size='12pt', 
       text_color=GRAY4)

# Add label axis
p.add_layout(Title(text="2014", 
                   offset=65,                    
                   text_color=GRAY4,
                   text_font=FONT,
                   text_font_size='14pt',
                   text_font_style='normal'), 
             "below")

# Add footing legend
p.add_layout(Title(text='Data source: XYZ Dashboard, as of 12/31/2014 | A detalled analysis of tickets processed per person and time to resolve', 
                   offset=0,                    
                   text_color=GRAY3,
                   text_font=FONT,
                   text_font_size='10pt',
                   text_font_style='normal'), 
             place="below")

p.add_layout(Title(text='issues was undertaken to inform this request and can be provided if needed.', 
                   offset=0,                    
                   text_color=GRAY3,
                   text_font=FONT,
                   text_font_size='10pt',
                   text_font_style='normal'), 
             place="below")

# Elements attributes

# Modify title attributes             
p.title.offset = 0
p.title.text_color = GRAY4
p.title.text_font=FONT
p.title.text_font_size = '23pt'


# Modify X axis attributes             
p.xaxis.bounds = (1.3, 13)
p.xaxis.fixed_location = 0  
p.xaxis.axis_line_color = GRAY8
p.xaxis.major_label_text_color = GRAY4            
p.xaxis.major_label_text_font = FONT
p.xaxis.major_label_text_font_size = '12pt'             
p.xaxis.major_tick_line_color = None
p.xgrid.grid_line_color = None

# Modify Y axis attributes
p.yaxis.bounds = (0, 300)
p.yaxis.fixed_location = 'Jan'           
p.yaxis.axis_line_color = GRAY8
p.yaxis.major_label_standoff = 10
p.yaxis.major_label_text_color = GRAY4             
p.yaxis.major_label_text_font = FONT
p.yaxis.major_label_text_font_size = '12pt'        
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_line_color = GRAY8
p.yaxis.minor_tick_line_color = None
p.ygrid.grid_line_color=None

# Convert the figure to png
p.background_fill_color = None
p.border_fill_color = None
p.outline_line_color = None

show(p)